# Clean FAOSTAT Data
---

In [2]:
# Load dependencies
import pandas as pd

## Load Datasets
---

In [3]:
# List available datasets
QCL_df = pd.read_csv('data/faostat/QCL_init.csv')
RL_df = pd.read_csv('data/faostat/RL_init.csv')
hist_df = pd.read_csv('data/faostat/history.csv')

In [4]:
hist_df.columns

Index(['Former', 'Succession', 'Exists'], dtype='object')

## Clean QCL 'Area': Project historic data onto current countries
---

In [5]:
# add historic data to existing countries
for i,row in hist_df.iterrows():
    if row['Former'] in QCL_df.Area.unique():
        for n in row['Succession']:
            if n in QCL_df.Area.unique():
                QCL_df.loc[n,'Value'] += QCL_df.loc[row['Former'],'Value']
        # remove countries that no longer exist
        if not row['Exists']: QCL_df.drop(QCL_df.index[QCL_df.Area==row['Former']], inplace = True)

In [6]:
# remove 'China' ie. china (total) from data
QCL_df.drop(QCL_df.index[QCL_df.Area=='China'], inplace = True)

## Clean QCL 'Item': Merge 'Maize (corn)' and 'Green corn (maize)'
---

In [7]:
QCL_df.columns

Index(['Area', 'Item', 'Year', 'Unit', 'Value'], dtype='object')

In [8]:
QCL_df.Item.unique()

array(['Maize (corn)', 'Rice', 'Cattle and Buffaloes', 'Soya beans',
       'Cocoa beans', 'Coffee, green', 'Oil palm fruit',
       'Green corn (maize)'], dtype=object)

In [9]:
QCL_df.count()

Area     47868
Item     47868
Year     47868
Unit     47868
Value    47868
dtype: int64

In [10]:
QCL_corn = QCL_df.loc[QCL_df.Item.isin(['Maize (corn)','Green corn (maize)'])]
QCL_corn.count()

Area     12035
Item     12035
Year     12035
Unit     12035
Value    12035
dtype: int64

In [11]:
QCL_else = QCL_df.loc[~QCL_df.Item.isin(['Maize (corn)','Green corn (maize)'])]
QCL_else.count()

Area     35833
Item     35833
Year     35833
Unit     35833
Value    35833
dtype: int64

In [12]:
QCL_corn.loc[:,'AreaYear'] = [x+str(y) for x,y in zip(QCL_corn.Area,QCL_corn.Year)]
QCL_corn.tail(1)

C:\Users\b25ch\AppData\Local\Temp\ipykernel_19556\1435106808.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  QCL_corn.loc[:,'AreaYear'] = [x+str(y) for x,y in zip(QCL_corn.Area,QCL_corn.Year)]


,Area,Item,Year,Unit,Value,AreaYear
48816,Zimbabwe,Maize (corn),2022,ha,1000000,Zimbabwe2022


In [13]:
QCL_corns = QCL_corn.groupby('AreaYear').sum()
QCL_corns.tail(1)

,Area,Item,Year,Unit,Value
AreaYear,,,,,
Zimbabwe2022,ZimbabweZimbabwe,Green corn (maize)Maize (corn),4044,haha,1001819


In [14]:
QCL_corna = QCL_corns.loc[QCL_corns.Unit=='ha']
QCL_corna.loc[:,'Item'] = 'Green corn/Maize'
QCL_corna = QCL_corna.reset_index()[['Area','Item','Year','Unit','Value']]

In [15]:
QCL_cornb = QCL_corns.loc[QCL_corns.Unit=='haha']
QCL_cornb.loc[:,'Area'] = [s[0:len(s)-4] for s in QCL_cornb.index]
QCL_cornb.loc[:,'Year'] = [int(s[len(s)-4:]) for s in QCL_cornb.index]
QCL_cornb.loc[:,'Unit'] = 'ha'
QCL_cornb.loc[:,'Item'] = 'Green corn/Maize'
QCL_cornb = QCL_cornb.reset_index()[['Area','Item','Year','Unit','Value']]

In [16]:
QCL_cornb

,Area,Item,Year,Unit,Value
0,Australia,Green corn/Maize,1961,ha,75680
1,Australia,Green corn/Maize,1962,ha,86151
2,Australia,Green corn/Maize,1963,ha,85776
3,Australia,Green corn/Maize,1964,ha,88009
4,Australia,Green corn/Maize,1965,ha,87123
...,...,...,...,...,...
2429,Zimbabwe,Green corn/Maize,2018,ha,1156828
2430,Zimbabwe,Green corn/Maize,2019,ha,993170
2431,Zimbabwe,Green corn/Maize,2020,ha,1020723
2432,Zimbabwe,Green corn/Maize,2021,ha,959595


In [17]:
QCL_clean = pd.concat([QCL_else,QCL_corna,QCL_cornb],ignore_index=True)
QCL_clean.tail(1)

,Area,Item,Year,Unit,Value
45433,Zimbabwe,Green corn/Maize,2022,ha,1001819


## Export QCL to csv
---

In [18]:
QCL_clean.to_csv('data/faostat/QCL_clean.csv',index=False)

## Clean RL 'Area': Project historic data onto current countries
---

In [19]:
for i,row in hist_df.iterrows():
    if row['Former'] in RL_df.Area.unique():
        for n in row['Succession']:
            if n in RL_df.Area.unique():
                RL_df.loc[n,'Value'] += RL_df.loc[row['Former'],'Value']
        if not row['Exists']: RL_df.drop(RL_df.index[RL_df.Area==row['Former']], inplace = True)

In [20]:
# remove 'China' ie. china (total) from data
RL_df.drop(RL_df.index[RL_df.Area=='China'], inplace = True)

## Clean RL 'Area': 
---

In [21]:
RL_df.tail(1)

,Area,Item,Year,Unit,Value
75131,Zimbabwe,Primary Forest,2017,1000 ha,801.0


In [22]:
RL_df.Item.unique()

array(['Land area', 'Cropland', 'Permanent meadows and pastures',
       'Forest land', 'Naturally regenerating forest', 'Planted Forest',
       'Other land', 'Primary Forest', 'Farm buildings and Farmyards',
       'Land used for aquaculture'], dtype=object)

In [23]:
RL_df.loc[:,'AreaYear'] = [x+str(y) for x,y in zip(RL_df.Area,RL_df.Year)]
RL_df.tail(1)

,Area,Item,Year,Unit,Value,AreaYear
75131,Zimbabwe,Primary Forest,2017,1000 ha,801.0,Zimbabwe2017


In [54]:
# Land Area = Cropland + Permanent meadows and pastures + Forest land + 
#   Other land(barren land and desert area, urban land, infrastructure) + Farm buildings and Farmyards
land_types = ['Land area','Cropland','Permanent meadows and pastures','Forest land','Other land', 'Farm buildings and Farmyards']

In [55]:
RL_dense = RL_df.loc[RL_df.Item.isin(land_types)][['Item','Value','AreaYear']].groupby('AreaYear').agg(list)

In [56]:
RL_dense['Unaccounted'] = [ x[0] - sum(x[1:len(x)]) for x in RL_dense['Value']]

In [57]:
RL_dense

,Item,Value,Unaccounted
AreaYear,,,
Afghanistan1961,"[Land area, Cropland, Permanent meadows and pa...","[65223.0, 7750.0, 30000.0]",27473.0
Afghanistan1962,"[Land area, Cropland, Permanent meadows and pa...","[65223.0, 7800.0, 30000.0]",27423.0
Afghanistan1963,"[Land area, Cropland, Permanent meadows and pa...","[65223.0, 7850.0, 30000.0]",27373.0
Afghanistan1964,"[Land area, Cropland, Permanent meadows and pa...","[65223.0, 7905.0, 30000.0]",27318.0
Afghanistan1965,"[Land area, Cropland, Permanent meadows and pa...","[65223.0, 7910.0, 30000.0]",27313.0
...,...,...,...
Zimbabwe2018,"[Land area, Cropland, Permanent meadows and pa...","[38685.0, 3367.6304, 12100.0, 17536.72, 5680.6...",0.0
Zimbabwe2019,"[Land area, Cropland, Permanent meadows and pa...","[38685.0, 3187.6768, 12100.0, 17490.65, 5906.6...",0.0
Zimbabwe2020,"[Land area, Cropland, Permanent meadows and pa...","[38685.0, 3278.8634, 12100.0, 17444.58, 5861.5...",0.0


In [59]:
RL_dense.loc[RL_dense.Unaccounted>1].count()

Item           5917
Value          5917
Unaccounted    5917
dtype: int64

In [60]:
for t in land_types: RL_dense[t] = 0.0

In [61]:
for name in RL_dense.index:
    for i in range(0,len(RL_dense.Item[name])):
        RL_dense.loc[name,RL_dense.Item[name][i]] = RL_dense.Value[name][i]

In [71]:
RL_clean = pd.DataFrame()
RL_clean['Area'] = [s[0:len(s)-4] for s in RL_dense.index]
RL_clean['Year'] = [int(s[len(s)-4:]) for s in RL_dense.index]
for type in land_types: RL_clean[type] = [*RL_dense[type]]
RL_clean['Unaccounted'] = [*RL_dense['Unaccounted']]

In [72]:
RL_clean

,Area,Year,Land area,Cropland,Permanent meadows and pastures,Forest land,Other land,Farm buildings and Farmyards,Unaccounted
0,Afghanistan,1961,65223.0,7750.0000,30000.0,0.00,0.0000,0.0,27473.0
1,Afghanistan,1962,65223.0,7800.0000,30000.0,0.00,0.0000,0.0,27423.0
2,Afghanistan,1963,65223.0,7850.0000,30000.0,0.00,0.0000,0.0,27373.0
3,Afghanistan,1964,65223.0,7905.0000,30000.0,0.00,0.0000,0.0,27318.0
4,Afghanistan,1965,65223.0,7910.0000,30000.0,0.00,0.0000,0.0,27313.0
...,...,...,...,...,...,...,...,...,...
13313,Zimbabwe,2018,38685.0,3367.6304,12100.0,17536.72,5680.6496,0.0,0.0
13314,Zimbabwe,2019,38685.0,3187.6768,12100.0,17490.65,5906.6732,0.0,0.0
13315,Zimbabwe,2020,38685.0,3278.8634,12100.0,17444.58,5861.5566,0.0,0.0
13316,Zimbabwe,2021,38685.0,3136.4378,12100.0,17398.51,6050.0522,0.0,0.0


## Export RL Data
---

In [73]:
RL_clean.to_csv('data/faostat/RL_clean.csv',index=False)